# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Transform and Load the Processed Data into Apache Cassandra 

## The processed data is contained in the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns:

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The screenshot below shows how the denormalized data should appear in the <font color=red>**event_datafile_new.csv**</font> after the code in __Part I__ is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Create a Cluster

In [5]:
# This should make a connection to a Cassandra instance on your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
session.execute("""
CREATE KEYSPACE IF NOT EXISTS sparkifyks 
WITH REPLICATION = 
{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkifyks')

## Create queries to ask the following three questions of the data, beginning with CREATE TABLE statements.

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

The data fields required here are artist name (artist), song title (song), song duration (length). The artist and song name values vary significantly in size and character type (including some special characters), so allowing for a free-form UTF-8 encoded string was the best choice for these.  For the song duration (length) I considered specifying the numerical precision and scale, as the values seemed somewhat constant, but decided a variable-precision decimal would be better as the values could fluctuate.   

For this query, the known values used to identify the requisite records are sessionId and itemInSession, so these should be used in the PRIMARY KEY to create the table.  The sessionId value is the PARTITION KEY because each user login session must contain one or more songs (itemInSession), i.e. the songs are grouped by sessionId (a playlist).  The itemInSession value is the CLUSTERING KEY because the song(s) played during a user login session (playlist) are sorted by it's "item" number.

In [9]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_duration "
query = query + "(sessionId int, itemInSession smallint, artist text, \
song text, length decimal, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

The data fields required for this table are artist name (artist), song title (song), login session (sessionId), song play in session (itemInSession) and user name (firstName, lastName). The artist and song name values vary significantly in size and character type (including some special characters), so allowing for a free-form UTF-8 encoded string was the best choice for these.  The userId and sessionId values could grow significantly over time, so a 32-bit int seemed appropriate.  The itemInSession numerical values will be persistently small so a 2-byte smallint data type was chosen.

Here, the known values are userId and sessionId and so these should appear in the PRIMARY KEY.  Because both values must be used to identify a unique record, they should comprise a COMPOSITE KEY for the PARTITION KEY.  The query requires that the songs in the session (playlist) be in sorted order by itemInSession, so this field is included in the PRIMARY KEY as a CLUSTERING KEY. 

In [10]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "(userId int, sessionId int, itemInSession smallint, \
artist text, song text, userFirstName varchar, userLastName varchar, \
PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In this query the identifying value is the song name, so this is included as the PARTITION KEY in the PRIMARY KEY.  Because it is possible that users may have the same first and last names, it is also necessary to include the userId value in the query, and to include this value as the CLUSTERING KEY in the PRIMARY KEY.

In [11]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_user "
query = query + "(song text, userFirstName varchar, userLastName varchar, userId int, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Read the data from the source CSV file <font color=red>**event_datafile_new.csv**</font> and load it into the associated fields in each of the three tables created above.

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    ## Read each line in the CSV file and execute an INSERT CQL statement for each of the three tables.
    for line in csvreader:
        ## Form the query and assign to a variable
        query = "INSERT INTO song_playlist_duration (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign the line elements to their associated table columns in the query, and execute
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], np.float(line[5])))
        except Exception as e:
            print(e)
        ## Form the query and assign to a variable
        query = "INSERT INTO song_playlist_session (userId, sessionId, itemInSession, artist, song, userFirstName, userLastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign the line elements to their associated table columns in the query, and execute
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)
        ## Form the query and assign to a variable
        query = "INSERT INTO song_playlist_user (song, userId, userFirstName, userLastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Assign the line elements to their associated table columns in the query, and execute
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)

#### Execute a SELECT statement on each of the three tables to verify that the data have been inserted.

In [13]:
## Select values from all columns but limit the result set to five records
query = "SELECT * FROM song_playlist_duration LIMIT 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
## Include separator between result sets
print("--- SAMPLE DATA FROM song_playlist_duration TABLE --- \n")
for row in rows:
    print(row)
    
## Select values from all columns but limit the returned data set to five records
query = "SELECT * FROM song_playlist_session LIMIT 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
## Include separator between result sets
print("\n --- SAMPLE DATA FROM song_playlist_session TABLE --- \n")
for row in rows:
    print(row)

## Select values from all columns but limit the returned data set to five records
query = "SELECT * FROM song_playlist_user LIMIT 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
## Include separator between result sets
print("\n--- SAMPLE DATA FROM song_playlist_user TABLE --- \n")
for row in rows:
    print(row)

--- SAMPLE DATA FROM song_playlist_duration TABLE --- 

Row(sessionid=23, iteminsession=0, artist='Regina Spektor', length=Decimal('191.08526'), song='The Calculation (Album Version)')
Row(sessionid=23, iteminsession=1, artist='Octopus Project', length=Decimal('250.95791'), song='All Of The Champs That Ever Lived')
Row(sessionid=23, iteminsession=2, artist='Tegan And Sara', length=Decimal('180.06159'), song='So Jealous')
Row(sessionid=23, iteminsession=3, artist='Dragonette', length=Decimal('153.39057'), song='Okay Dolores')
Row(sessionid=23, iteminsession=4, artist='Lil Wayne / Eminem', length=Decimal('229.58975'), song='Drop The World')

 --- SAMPLE DATA FROM song_playlist_session TABLE --- 

Row(userid=58, sessionid=768, iteminsession=0, artist='System of a Down', song='Sad Statue', userfirstname='Emily', userlastname='Benson')
Row(userid=58, sessionid=768, iteminsession=1, artist='Ghostland Observatory', song='Stranger Lover', userfirstname='Emily', userlastname='Benson')
Row(useri

### Execute SELECT statements to fulfill the result sets required by the three questions posed above.

__Query 1__:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [14]:
query = "SELECT artist, song, length, sessionId, itemInSession FROM song_playlist_duration WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## Print a header row of column names for better result set readability, including column delineation
print(rows.column_names[0], "|", rows.column_names[1], "|", rows.column_names[2], "|", rows.column_names[3], "|", rows.column_names[4])

## Print each row from the result set
for row in rows:
    print(row.artist, "|", row.song, "|", row.length, "|", row.sessionid, "|", row.iteminsession)

artist | song | length | sessionid | iteminsession
Faithless | Music Matters (Mark Knight Dub) | 495.3073 | 338 | 4


__Query 2__: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [15]:
query = "SELECT artist, song, userFirstName, userLastName, itemInSession FROM song_playlist_session WHERE userId=10 AND sessionId=182 ORDER BY itemInSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## Print a header row of column names for better result set readability, including column delineation
print(rows.column_names[0], "|", rows.column_names[1], "|", rows.column_names[2], "|", rows.column_names[3], "|", rows.column_names[4])

## Print each row from the result set
for row in rows:
    print(row.artist, "|", row.song, "|", row.userfirstname, "|", row.userlastname, "|", row.iteminsession)

artist | song | userfirstname | userlastname | iteminsession
Down To The Bone | Keep On Keepin' On | Sylvie | Cruz | 0
Three Drives | Greece 2000 | Sylvie | Cruz | 1
Sebastien Tellier | Kilometer | Sylvie | Cruz | 2
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie | Cruz | 3


__Query 3__: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
query = "SELECT song, userId, userFirstName, userLastName FROM song_playlist_user WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
## Print a header row of column names for better result set readability, including column delineation
print(rows.column_names[0], "|", rows.column_names[1], "|", rows.column_names[2], "|", rows.column_names[3])

## Print each row from the result set
for row in rows:
    print(row.song, "|", row.userid, "|", row.userfirstname, "|", row.userlastname)

song | userid | userfirstname | userlastname
All Hands Against His Own | 29 | Jacqueline | Lynch
All Hands Against His Own | 80 | Tegan | Levine
All Hands Against His Own | 95 | Sara | Johnson


### Drop the tables before closing out the session

In [17]:
query = "DROP TABLE IF EXISTS song_playlist_duration"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS song_playlist_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection

In [18]:
session.shutdown()
cluster.shutdown()